<a href="https://colab.research.google.com/github/emmiduh/ai_agents_mcp_examples/blob/main/ai_agents_mcp_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q google-generativeai

import google.generativeai as genai
from google.colab import userdata


API_KEY = userdata.get('GEMINI_API_KEY')

if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Please add it in Colab Secrets.")

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

print("Welcome to your AI Assistant. Type 'goodbye' to quit.")

while True:
    prompt = input("You: ")
    if prompt.lower() == "goodbye":
        print("AI Assistant: Goodbye!")
        break
    response = model.generate_content(prompt)

    print("AI Assistant:", response.text)

Welcome to your AI Assistant. Type 'goodbye' to quit.
You: goodbye
AI Assistant: Goodbye!
